In [643]:
import numpy as np
import pandas as pd
import re
import datetime

# Code

In [644]:
data = 'https://www.ndbc.noaa.gov/data/realtime2/46029.spec'
article_read = pd.read_csv(data, sep='\s+', delimiter=' ', encoding="utf-8", skipinitialspace=True)

In [645]:
article_read = article_read.drop(0)

In [646]:
article_read.head()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
1,2019,03,22,00,00,1.8,1.8,19.0,0.4,3.8,NW,SSW,SWELL,7.6,315
2,2019,03,21,23,00,1.8,1.7,10.8,0.5,4.0,WSW,SSW,SWELL,7.4,252
3,2019,03,21,22,00,1.8,1.7,12.9,0.6,4.2,WSW,SSW,SWELL,7.2,243
4,2019,03,21,21,00,1.7,1.6,12.1,0.6,4.8,W,SSW,SWELL,7.2,280
5,2019,03,21,20,00,1.9,1.8,12.1,0.6,3.8,WSW,S,SWELL,7.4,245


In [647]:
short_table = article_read.drop(article_read.iloc[:,3:6,], axis=1)

In [648]:
short_table = short_table.drop(['STEEPNESS', 'APD', 'MWD'], axis=1)

In [649]:
d = datetime.date.today().strftime('%Y-%m-%d')
d

'2019-03-21'

In [650]:
short_table = short_table.rename(columns={'#YY': 'YY'})

In [651]:
date = (short_table.YY + '-' + short_table.MM + '-' + short_table.DD)
short_table['Date'] = date

In [652]:
data_table = short_table.drop(short_table.columns[[0,1,2]], axis=1)
data_table = data_table[data_table['Date']==d]

In [653]:
data_table['SwH'] = data_table.SwH.astype(float)
data_table['SwP'] = data_table.SwP.astype(float)
data_table['WWH'] = data_table.WWH.astype(float)
data_table['WWP'] = data_table.WWP.astype(float)

In [654]:
data_table.dtypes

SwH     float64
SwP     float64
WWH     float64
WWP     float64
SwD      object
WWD      object
Date     object
dtype: object

In [655]:
wave = data_table.SwH * data_table.SwP
data_table['Wave'] = wave

In [656]:
article_read.WWD.unique()

array(['SSW', 'S', 'SW', 'ESE', 'SSE', 'SE', 'E', 'WSW', 'ENE', 'N', 'W',
       'NNE', 'WNW', 'NE', 'NW', 'NNW'], dtype=object)

In [657]:
wwd = pd.DataFrame({"WWD": [ 'SW', 'SSW', 'S', 'ESE', 'SSE', 'SE', 'E', 'WSW', 'ENE',
       'N', 'W', 'NNE', 'WNW', 'NE', 'NW', 'NNW' ], "scoreWWD": [1, 1, 3 ,1 ,1 ,4 ,5 ,1 ,1 ,1 ,1 ,1 , 1, 4,1,1]})
wwd.head()

,WWD,scoreWWD
0,SW,1
1,SSW,1
2,S,3
3,ESE,1
4,SSE,1


In [658]:
data_table =pd.merge(data_table, wwd)
data_table.head()

,SwH,SwP,WWH,WWP,SwD,WWD,Date,Wave,scoreWWD
0,1.7,10.8,0.5,4.0,WSW,SSW,2019-03-21,18.36,1
1,1.7,12.9,0.6,4.2,WSW,SSW,2019-03-21,21.93,1
2,1.6,12.1,0.6,4.8,W,SSW,2019-03-21,19.36,1
3,1.8,10.8,0.6,3.7,W,SSW,2019-03-21,19.44,1
4,2.1,12.1,0.5,3.6,W,SSW,2019-03-21,25.41,1


In [659]:
def scoreWave(column): 
    '''
    This is a test:
    >>> scoreWave(31)
    5
    
    This is a test:
    >>> scoreWave(30)
    4

    This is a test:
    >>> scoreWave(11)
    2

    This is a test:
    >>> scoreWave(5)
    1
    '''
    
    if column > 30 :
        score = 5
    elif column >= 25 :
        score = 4
    elif column >= 20 :
        score = 3
    elif column >= 11 :
        score = 2
    else:
        score =1
    return score 

In [660]:
def scoreSwp(column):
    '''
    This is a test:
    >>> scoreSwp(16)
    5
    
    This is a test:
    >>> scoreSwp(15)
    4
    
    This is a test:
    >>> scoreSwp(10)
    4
    
    This is a test:
    >>> scoreSwp(5)
    1
    '''
        
    if column >= 16 :
        score = 5
    elif column >= 12 :
        score = 4
    elif column >= 10 :
        score = 4
    else:
        score =1
    return score 

In [661]:
data_table['scoreWave'] = data_table['Wave'].map(scoreWave)

In [662]:
data_table['scoreSwP'] = data_table['SwP'].map(scoreSwp)

In [663]:
data_length = len(data_table)-1
data_length

23

In [664]:
data_table

,SwH,SwP,WWH,WWP,SwD,WWD,Date,Wave,scoreWWD,scoreWave,scoreSwP
0,1.7,10.8,0.5,4.0,WSW,SSW,2019-03-21,18.36,1,2,4
1,1.7,12.9,0.6,4.2,WSW,SSW,2019-03-21,21.93,1,3,4
2,1.6,12.1,0.6,4.8,W,SSW,2019-03-21,19.36,1,2,4
3,1.8,10.8,0.6,3.7,W,SSW,2019-03-21,19.44,1,2,4
4,2.1,12.1,0.5,3.6,W,SSW,2019-03-21,25.41,1,4,4
5,2.1,12.1,0.5,5.0,W,SSW,2019-03-21,25.41,1,4,4
6,2.0,13.8,0.5,4.5,W,SSW,2019-03-21,27.60,1,4,4
7,1.9,12.9,0.5,3.3,W,SSW,2019-03-21,24.51,1,3,4
8,1.9,12.9,0.8,3.3,W,SSW,2019-03-21,24.51,1,3,4
9,1.9,13.8,0.7,3.3,WNW,SSW,2019-03-21,26.22,1,4,4


In [665]:
totalscore = data_table['scoreWWD'].sum()/(len(data_table)-1)
totalscore

1.565217391304348

In [666]:
day_score = round(totalscore)
day_score

2.0

# Test

The test is inside the function marked by ''' , this is space sensative so data must match the alighment.

In [667]:
data_length

23

In [668]:
data_table.apply(np.sum, axis=0)

SwH                                                       45.8
SwP                                                      302.2
WWH                                                       13.4
WWP                                                      102.5
SwD                 WSWWSWWWWWWWWWNWWWSWWWSWWWSWWWWSWWNWWWWNWW
WWD          SSWSSWSSWSSWSSWSSWSSWSSWSSWSSWSSWSSSSSSSWSWSWS...
Date         2019-03-212019-03-212019-03-212019-03-212019-0...
Wave                                                    577.75
scoreWWD                                                    36
scoreWave                                                   80
scoreSwP                                                    96
dtype: object

In [601]:
data_table[['scoreWWD','scoreWave','scoreSwP']].mean()

scoreWWD     1.500000
scoreWave    3.333333
scoreSwP     4.000000
dtype: float64

In [602]:
def total(column): 
    '''
    This is a test:
    >>> total('scoreWWD') 
    36
    
     This is a test:
    >>> total('scoreWave') 
    80
    
    This is a test:
    >>> total('scoreSwP') 
    96
    '''
    score = data_table[column].sum()
    return round(score)

In [603]:
def total_avg(column): 
    '''
    This is a test:
    >>> total_avg('scoreWWD') 
    2.0
    
    This is a test:
    >>> total_avg('scoreWave') 
    3.0
    
    This is a test:
    >>> total_avg('scoreSwP') 
    4.0
    '''
    score = data_table[column].sum()/(data_length)
    return round(score)

In [604]:
total('scoreWWD')

36

In [605]:
total_avg('scoreWWD')

2.0

In [606]:
total_avg('scoreWave')

3.0

In [607]:
total_avg('scoreSwP')

4.0

In [608]:
import doctest
doctest.testmod(verbose=True)

Trying:
    scoreSwp(16)
Expecting:
    5
ok
Trying:
    scoreSwp(15)
Expecting:
    4
ok
Trying:
    scoreSwp(10)
Expecting:
    4
ok
Trying:
    scoreSwp(5)
Expecting:
    1
ok
Trying:
    scoreWave(31)
Expecting:
    5
ok
Trying:
    scoreWave(30)
Expecting:
    4
ok
Trying:
    scoreWave(11)
Expecting:
    2
ok
Trying:
    scoreWave(5)
Expecting:
    1
ok
Trying:
    total('scoreWWD') 
Expecting:
    36
ok
Trying:
    total('scoreWave') 
Expecting:
    80
ok
Trying:
    total('scoreSwP') 
Expecting:
    96
ok
Trying:
    total_avg('scoreWWD') 
Expecting:
    2.0
ok
Trying:
    total_avg('scoreWave') 
Expecting:
    3.0
ok
Trying:
    total_avg('scoreSwP') 
Expecting:
    4.0
ok
1 items had no tests:
    __main__
4 items passed all tests:
   4 tests in __main__.scoreSwp
   4 tests in __main__.scoreWave
   3 tests in __main__.total
   3 tests in __main__.total_avg
14 tests in 5 items.
14 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=14)

# UI

In [641]:
import ipywidgets as widgets

In [642]:
condition_today = data_table[['scoreWWD','scoreWave','scoreSwP']].mean()
condition_today

KeyError: "['scoreWWD' 'scoreWave' 'scoreSwP'] not in index"

In [ ]:
var_widget = widgets.DatePicker(
    description='Pick a Date',
    value = datetime.date.today(),
    disabled=False
)
var_widget